In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 27.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install transformers

In [3]:
import pandas as pd

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, BertModel
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig, DistilBertModel
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM
from transformers import LongformerConfig, LongformerTokenizer, LongformerModel

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics.cluster import contingency_matrix

import shutil
import torch
import json
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(42)


In [4]:
def pre_process_data(df, test_proportion, train_size):

    size_df = df.shape[0]
    df = shuffle(df, random_state = 42)
    X = df[["FQText"]]
    y = df.drop(["FQText"], axis=1).astype(np.float32)
    print(y.info())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_proportion, shuffle=True, random_state=42)

    df_train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
    df_test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)
    df_labels = list(y.columns)

    print(df_labels)

    train_df = df_train.sample(frac=train_size, random_state=42).reset_index(drop=True)
    val_df = df_train.drop(train_df.index).reset_index(drop=True)

    print("Total amount of data: {}".format(size_df))
    print("Number of rows used to TRAIN: {}".format(train_df.shape[0]))
    print("Number of rows used to VALIDATE: {}".format(val_df.shape[0]))
    print("Number of rows used to TEST: {}".format(df_test.shape[0]))

    return train_df, val_df, df_test, df_labels

In [5]:
def set_hyperparams(hp_dictionary):

    MAX_LEN = hp_dictionary["MAX_LEN"]
    TRAIN_BATCH_SIZE = hp_dictionary["TRAIN_BATCH_SIZE"]
    VALID_BATCH_SIZE = hp_dictionary["VALID_BATCH_SIZE"]
    EPOCHS = hp_dictionary["EPOCHS"]
    LEARNING_RATE = hp_dictionary["LEARNING_RATE"]

    return MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE

In [6]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [7]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['FQText']
        # self.labels = list(df.columns)[2:] # list of the target values
        self.targets = self.df[df_labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True, # do we actually need special tokens ??
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'  # pytorch tensors
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

CAMBIAR EL PROBLEM TYPE AQUÍ

In [8]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        # self.bert_model = LongformerModel.from_pretrained('allenai/longformer-base-4096', return_dict=True, problem_type="multi_label_classification")
        self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True, problem_type="multi_label_classification")

        self.dropout = torch.nn.Dropout(0.3) # why 0.3?
        self.linear = torch.nn.Linear(768, len(df_labels)) # have to changet he n of possible labels here

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )

        # the issue is that bert gets size 6 here?
        output_dropout = self.dropout(output.pooler_output)
        # print(output_dropout)
        output = self.linear(output_dropout)
        return output

In [9]:
def loss_fn(outputs, targets):
    # print(outputs, targets)
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

## Train Function

In [67]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path, df_labels):
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf

  val_targets_study= []
  val_outputs_study= [] # I am going to extend all the validation probabilities to determine what the prob threshold should be

  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        # print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        """targets are the y array of the original data.
        """

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)



    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    val_targets = []
    val_outputs = []

    sigmoid = torch.nn.Sigmoid()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(outputs.cpu().detach().numpy().tolist())

        probs = sigmoid(torch.Tensor(outputs))
        # print(probs)

        val_targets_study.append(targets.cpu().detach().numpy().tolist())
        val_outputs_study.append(probs.cpu().detach().numpy().tolist())

        # outputs = model(ids.squeeze(), mask.squeeze(), token_type_ids.squeeze())



      #       metrics_targets.extend(torch.argmax(softm(targets), dim=1).cpu().detach().numpy().tolist())
      #       metrics_outputs.extend(torch.argmax(softm(outputs), dim=1).cpu().detach().numpy().tolist())

      # metrics_outputs = np.array(metrics_outputs, dtype=int)
      # val_f1 = f1_score(metrics_outputs, metrics_targets, average="weighted")
      # val_acc = accuracy_score(metrics_outputs, metrics_targets)
      # class_report = classification_report(metrics_outputs, metrics_targets, target_names = df_labels)

      # print((f"Accuracy: {val_f1}"))
      # print((f"F1 Score (Weighted): {val_f1}"))
      # print((f"Classification report: \n{class_report}"))



      print('############# Epoch {}: Validation End     #############'.format(epoch))

      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, True, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      # if valid_loss <= valid_loss_min:
      #   print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
      #   # save checkpoint as best model
      #   save_ckp(checkpoint, True, checkpoint_path, best_model_path)
      #   valid_loss_min = valid_loss



    print('############# Epoch {}  Metrics   #############\n\n'.format(epoch))
    metrics = multi_labels_metrics(val_outputs, val_targets)
    # print((f"EVAL METRICS: {metrics}\n"))

    print('############# Epoch {}  Done   #############\n'.format(epoch))
  return model, val_targets_study, val_outputs_study

In [47]:
# not used at the moment

# def compute_metrics(epoch, validation_loader):
#   model.eval()
#   metrics_targets = []
#   metrics_outputs = []
#   softm = torch.nn.Softmax(dim=1)
#   with torch.no_grad():
#         for _, data in enumerate(validation_loader, 0):
#               ids = data['input_ids'].to(device, dtype = torch.long)
#               mask = data['attention_mask'].to(device, dtype = torch.long)
#               token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#               targets = data['targets'].to(device, dtype = torch.float)

#               outputs = model(ids.squeeze(), mask.squeeze(), token_type_ids.squeeze())

#               metrics_targets.extend(torch.argmax(softm(targets), dim=1).cpu().detach().numpy().tolist())
#               metrics_outputs.extend(torch.argmax(softm(outputs), dim=1).cpu().detach().numpy().tolist())

#   return metrics_targets, metrics_outputs

TRAIN AND VALIDATION LOOP

# Determinants

In [48]:
df = pd.read_csv("nancy_determinants_grouped.csv")


GROUPED

In [49]:
df.rename(columns = {"Answer (English)": "FQText"}, inplace=True)

df.drop(['Déterminant', 'C', 'C\'', 'C\'F',
       'CF', 'CF\'', 'CLOB', 'CLOBF', 'E', 'EF', 'F', 'FC', 'FC\'', 'FCLOB',
       'FE', 'K', 'KAN', 'KOB', 'KP', 'Réponse (French)'], axis=1, inplace=True)

In [50]:
# df.columns

INDIVIDUAL

In [51]:
# df.rename(columns = {"Answer (English)": "FQText"}, inplace=True)

# df.drop(['Déterminant', 'color_sum', 'threat_sum', 'fading_sum',
#        'form_sum', 'kinesthetics_sum', 'Réponse (French)'], axis=1, inplace=True)

In [52]:
# df.columns

# Contents

In [53]:
# df = pd.read_csv("nancy_contents_grouped.csv")


GROUPED

In [54]:
# df.rename(columns = {"Answer (English)": "FQText"}, inplace=True)

# df.drop(['Contenu', '(A)', '(AD)', '(H)', '(HD)',
#        'A', 'ABS', 'AD', 'ALIM', 'ANAT', 'ARCH', 'ART', 'BOT', 'ELEM', 'FRAG',
#        'GÉO', 'H', 'HD', 'MQ', 'NAT', 'OBJ', 'PAYS', 'RADIO', 'SC', 'SCÈNE',
#        'SEX', 'SG', 'VÊT', 'Réponse (French)'], axis=1, inplace=True)

In [55]:
# df.columns

INDIVIDUAL

In [56]:
# df.rename(columns = {"Answer (English)": "FQText"}, inplace=True)

# df.drop(['Contenu', 'animal_sum', 'human_sum', 'abs_sum',
#        'food_sum', 'art_arch_sum', 'nature_sum', 'fragment_sum', 'geo_sum',
#        'object_sum', 'science_sum', 'graphic_sum', 'Réponse (French)'], axis=1, inplace=True)

In [57]:
# df.columns

# MODEL

In [58]:
hp_dictionary = {"MAX_LEN":512 ,
    "TRAIN_BATCH_SIZE": 16,
    "VALID_BATCH_SIZE": 16,
    "EPOCHS": 4,
    "LEARNING_RATE": 1e-05
}

CHANGE PROBLEM TYPE AQUI

In [59]:
# tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096', problem_type="multi_label_classification")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [60]:
MAX_LEN, TRAIN_BATCH_SIZE, VALID_BATCH_SIZE, EPOCHS, LEARNING_RATE = set_hyperparams(hp_dictionary)

test_proportion = 0.1
test_size = 0.85

train_df, val_df, df_test, df_labels  = pre_process_data(df, test_proportion, test_size)


train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BERTClass()
model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

proba_threshold = 0.5


ckpt_path = "curr_ckpt"
best_model_path = "best_model.pt"

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 266 to 102
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   color_sum         380 non-null    float32
 1   threat_sum        380 non-null    float32
 2   fading_sum        380 non-null    float32
 3   form_sum          380 non-null    float32
 4   kinesthetics_sum  380 non-null    float32
dtypes: float32(5)
memory usage: 10.4 KB
None
['color_sum', 'threat_sum', 'fading_sum', 'form_sum', 'kinesthetics_sum']
Total amount of data: 380
Number of rows used to TRAIN: 291
Number of rows used to VALIDATE: 51
Number of rows used to TEST: 38


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Counting the labels present in each train/test/val batch


In [61]:
# def count_instances(y):
#   total_labels = 0
#   for col in y:
#       if col == "FQText":
#           continue
#       else:
#           sum_examples = y[col].sum()
#           print(col, sum_examples)

In [62]:
# y_val = val_df.drop("FQText", axis=1)
# y_train = train_df.drop("FQText", axis=1)
# y_test = df_test.drop("FQText", axis=1)


In [63]:
# print("train class distribution".upper())
# print("")
# print(count_instances(y_train))
# print("total size of the test data", len(y_train))
# print("\nvalidation class distribution".upper())
# print("")
# print(count_instances(y_val))
# print("total size of the test data", len(y_val))
# print("\ntest class distribution".upper())
# print("")
# print(count_instances(y_test))
# print("total size of the test data", len(y_test))

https://discuss.huggingface.co/t/dataset-label-format-for-multi-label-text-classification/14998


https://colab.research.google.com/drive/1aue7x525rKy6yYLqqt-5Ll96qjQvpqS7#scrollTo=6d869uZsT9MH

https://discuss.huggingface.co/t/multi-class-using-dataset/8970/3

https://www.youtube.com/watch?v=ZYc9za75Chk

https://arxiv.org/abs/2004.05150

https://huggingface.co/docs/transformers/model_doc/longformer


https://www.evidentlyai.com/classification-metrics/classification-threshold  - precision recall curve to set prob thresholds, class separation quality

In [69]:
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction
import torch


def multi_labels_metrics(predictions, targets, threshold=0.355, df_labels=df_labels): # mess with threshold
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions)) #mapply activation function in the raw values

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = targets

  f1 = f1_score(y_true, y_pred, average = 'micro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
  hamming = hamming_loss(y_true, y_pred)
  class_report = classification_report(y_true, y_pred, target_names=df_labels)

  metrics = {
      "roc_auc": roc_auc,       # special multilabel metrics
      "hamming_loss": hamming,  # special multilabel metrics
      "f1": f1
  }
  print(metrics)
  print(class_report)
  return metrics

def compute_test_metrics(test_loader, val_outputs_study, val_targets_study):

  model.eval()
  metrics_targets = []
  metrics_outputs = []
  sigmoid = torch.nn.Sigmoid()
  with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
              ids = data['input_ids'].to(device, dtype = torch.long)
              mask = data['attention_mask'].to(device, dtype = torch.long)
              token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
              targets = data['targets'].to(device, dtype = torch.float)

              outputs = model(ids.squeeze(), mask.squeeze(), token_type_ids.squeeze())

              metrics_targets.extend(targets.cpu().detach().numpy().tolist())
              metrics_outputs.extend(outputs.cpu().detach().numpy().tolist())


              probs = sigmoid(torch.Tensor(outputs))
              # print(probs)

              val_targets_study.append(targets.cpu().detach().numpy().tolist())
              val_outputs_study.append(probs.cpu().detach().numpy().tolist())

  metrics = multi_labels_metrics(metrics_outputs, metrics_targets)
  print((f"EVAL METRICS: {metrics}"))

  return metrics, val_outputs_study, val_targets_study


## TRAIN

In [68]:
trained_model, val_targets_study, val_outputs_study = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path, df_labels)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
tensor([[0.4227, 0.1626, 0.2670, 0.4591, 0.3432],
        [0.4448, 0.1679, 0.2875, 0.4313, 0.3179],
        [0.4192, 0.1600, 0.2555, 0.4413, 0.3339],
        [0.4191, 0.1763, 0.2852, 0.4401, 0.3291],
        [0.4481, 0.1638, 0.2705, 0.5664, 0.3447],
        [0.4808, 0.1768, 0.2706, 0.5990, 0.3380],
        [0.4931, 0.1523, 0.2511, 0.4856, 0.2914],
        [0.4746, 0.1660, 0.2712, 0.5696, 0.3069],
        [0.4105, 0.1479, 0.2760, 0.4388, 0.3124],
        [0.4311, 0.1588, 0.2629, 0.5440, 0.3454],
        [0.4700, 0.1637, 0.2601, 0.5389, 0.3226],
        [0.4527, 0.1558, 0.2671, 0.4235, 0.3079],
        [0.4252, 0.1765, 0.2734, 0.5674, 0.3304],
        [0.4554, 0.1646, 0.2714, 0.5965, 0.3475],
        [0.4821, 0.1905, 0.2887, 0.4400, 0.3090],
        [0.4104, 0.1549, 0.2823, 0.5493, 0.3633]], device='cuda:0')
tensor([[0.4470, 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
tensor([[0.3428, 0.1534, 0.2238, 0.3461, 0.3710],
        [0.3800, 0.1538, 0.2455, 0.3299, 0.3410],
        [0.3487, 0.1356, 0.2007, 0.3458, 0.3277],
        [0.3671, 0.1613, 0.2455, 0.3411, 0.3270],
        [0.3372, 0.1290, 0.1883, 0.4307, 0.3090],
        [0.3984, 0.1219, 0.1802, 0.5026, 0.2751],
        [0.4140, 0.1314, 0.1870, 0.3204, 0.2759],
        [0.4149, 0.1231, 0.1890, 0.5118, 0.2481],
        [0.3497, 0.1464, 0.2358, 0.3121, 0.3238],
        [0.3457, 0.1318, 0.1858, 0.3846, 0.3080],
        [0.4023, 0.1192, 0.1710, 0.4342, 0.2605],
        [0.4181, 0.1423, 0.2304, 0.3078, 0.3055],
        [0.3608, 0.1293, 0.1851, 0.4855, 0.2636],
        [0.3408, 0.1255, 0.1781, 0.4554, 0.3079],
        [0.4302, 0.1514, 0.2249, 0.3547, 0.2857],
        [0.3241, 0.1253, 0.1954, 0.4249, 0.3371]], device='cuda:0')
tensor([[0.3754, 0.1398, 0.1799, 0.3611, 0.2975],
        [0.3642, 0.149

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
tensor([[0.3299, 0.1385, 0.1978, 0.3393, 0.3833],
        [0.3694, 0.1331, 0.2162, 0.3235, 0.3486],
        [0.3215, 0.1096, 0.1581, 0.3596, 0.3223],
        [0.3468, 0.1287, 0.2037, 0.3422, 0.3163],
        [0.3049, 0.0995, 0.1416, 0.5068, 0.2810],
        [0.3325, 0.1086, 0.1501, 0.5514, 0.2847],
        [0.3836, 0.1155, 0.1634, 0.3091, 0.3002],
        [0.3330, 0.1001, 0.1506, 0.5559, 0.2558],
        [0.3534, 0.1345, 0.2175, 0.2969, 0.3454],
        [0.3147, 0.1007, 0.1383, 0.4150, 0.2889],
        [0.3441, 0.0942, 0.1287, 0.4739, 0.2604],
        [0.4282, 0.1132, 0.1959, 0.2982, 0.2924],
        [0.2803, 0.1114, 0.1498, 0.5189, 0.2907],
        [0.3238, 0.1105, 0.1550, 0.5626, 0.3011],
        [0.3835, 0.1137, 0.1699, 0.3809, 0.2722],
        [0.2963, 0.1071, 0.1587, 0.4778, 0.3376]], device='cuda:0')
tensor([[0.3478, 0.1076, 0.1330, 0.3967, 0.2869],
        [0.3474, 0.120

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


############# Epoch 4: Training End     #############
############# Epoch 4: Validation Start   #############
tensor([[0.3173, 0.1196, 0.1729, 0.3259, 0.3810],
        [0.4107, 0.1163, 0.2089, 0.2839, 0.3345],
        [0.3120, 0.0786, 0.1183, 0.3726, 0.2738],
        [0.4064, 0.1191, 0.2116, 0.2943, 0.3052],
        [0.2865, 0.0812, 0.1211, 0.4807, 0.2430],
        [0.3928, 0.0996, 0.1385, 0.6459, 0.2365],
        [0.4519, 0.1006, 0.1548, 0.2794, 0.2842],
        [0.3489, 0.0792, 0.1275, 0.5804, 0.1984],
        [0.3900, 0.1172, 0.2094, 0.2546, 0.3320],
        [0.3557, 0.0836, 0.1225, 0.3777, 0.2535],
        [0.4416, 0.0883, 0.1223, 0.5632, 0.1989],
        [0.5024, 0.1015, 0.1946, 0.2748, 0.2769],
        [0.2833, 0.0838, 0.1175, 0.5214, 0.2266],
        [0.3519, 0.0953, 0.1391, 0.6454, 0.2461],
        [0.4669, 0.0947, 0.1557, 0.3693, 0.2276],
        [0.3066, 0.0765, 0.1285, 0.5641, 0.2686]], device='cuda:0')
tensor([[0.4127, 0.0786, 0.1072, 0.4412, 0.2161],
        [0.3961, 0.098

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TEST

In [70]:
test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [71]:
metrics, val_outputs_study, val_targets_study =  compute_test_metrics(test_data_loader, val_outputs_study, val_targets_study)

{'roc_auc': 0.6171428571428571, 'hamming_loss': 0.2894736842105263, 'f1': 0.43298969072164945}
                  precision    recall  f1-score   support

       color_sum       0.42      0.73      0.54        15
      threat_sum       0.00      0.00      0.00         6
      fading_sum       0.00      0.00      0.00         3
        form_sum       0.45      0.69      0.55        13
kinesthetics_sum       1.00      0.08      0.14        13

       micro avg       0.45      0.42      0.43        50
       macro avg       0.37      0.30      0.24        50
    weighted avg       0.50      0.42      0.34        50
     samples avg       0.43      0.49      0.43        50

EVAL METRICS: {'roc_auc': 0.6171428571428571, 'hamming_loss': 0.2894736842105263, 'f1': 0.43298969072164945}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Saving the study probabilities and targets

In [88]:
val_outputs_study = [value for row_proba in val_outputs_study for value in row_proba]
val_outputs_study = [value for row_proba in val_outputs_study for value in row_proba]

In [93]:
val_targets_study = [value for row_proba in val_targets_study for value in row_proba]
val_targets_study = [value for row_proba in val_targets_study for value in row_proba]

In [97]:
df_prob_target = pd.DataFrame([val_outputs_study, val_targets_study]).T

In [100]:
df_prob_target.rename(columns = {0:"Probability", 1:"Actual_target"}, inplace=True)

In [101]:
df_prob_target.to_csv("prob_target_study.csv", index=False)